## Implied volatility smiles from options

This notebook compares options on a stock and their implied volatilities with the observed volatilities in the past. The stock chosen is alphabet, since it conveniently does not pay any dividends and is fairly liquid. The analysis is done with historic closing data for GOOG up until 2020-12-29 and option price data at some time on that last day. Since I did not find freely available historic option price data, longitudinal analysis are unfortunately not possible.

In [14]:
from datetime import datetime
from io import StringIO
import os
import sys

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline.offline import iplot
import requests

sys.path.append(os.path.normpath(os.path.join(os.getcwd(), os.pardir)))
from methods.binom_trees import binom_tree, iv_put, iv_call, fv_american
from methods.implied_vol_options import implied_vol

dt_fmt = "%Y-%m-%d"
col_0 = "#1f77b4"
col_1 = "#ff7f0e"

In [2]:
# Download the data
goog_hist_csv = requests.get("https://raw.githubusercontent.com/o1i/hull/main/data/2020-12-27_GOOG.csv").content.decode("utf-8")
goog_opt_csv = requests.get("https://raw.githubusercontent.com/o1i/hull/main/data/20201229_goog_options_via_nasdaq.csv").content.decode("utf-8")

In [11]:
# Get Data into a usable format
hist = pd.read_csv(StringIO(goog_hist_csv))
opt = pd.read_csv(StringIO(goog_opt_csv))
opt["call"] = (opt["call_bid"] + opt["call_ask"]) / 2
opt["put"] = (opt["put_bid"] + opt["put_ask"]) / 2
s0 = hist.sort_values("Date")["Close"].iloc[-1]

# Time to expiry
def workdays_between(t1: datetime, t2: datetime) -> int:
    """Returns number of workdays between the two dates, excluding t1, including t2"""
    return sum(d.weekday() in [0, 1, 2, 3, 4] for d in pd.date_range(t1, t2, freq="d")[1:])

# Simplifying assumption: trading days are workdays (aka only weekends are not trading days)
t0 = datetime.strptime(max(hist["Date"]), dt_fmt)
t_n_map = {x: workdays_between(t0, x) for x in opt["date"].unique()}
opt["t_n"] = opt["date"].map(t_n_map) / 252
opt.sort_values(["date", "strike"])

# Risk-'free' rates from  T-Bills
# https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=billrates
r = 0.09

# Number of steps in the trees
n = 50

In [12]:
opt["call_sig"] = opt.apply(lambda x: implied_vol(binom_tree, 
                                                  x["call"], 
                                                  {
                                                      "s": s0, 
                                                      "k": x["strike"],
                                                      "r": r,
                                                      "t": x["t_n"],
                                                      "n": n,
                                                      "iv": iv_call,
                                                      "fv": fv_american
                                                  }), axis=1)
opt["put_sig"] = opt.apply(lambda x: implied_vol(binom_tree, 
                                                 x["put"], 
                                                 {
                                                     "s": s0, 
                                                     "k": x["strike"],
                                                     "r": r,
                                                     "t": x["t_n"],
                                                     "n": n,
                                                     "iv": iv_put,
                                                     "fv": fv_american
                                                 }), axis=1)

Expectations to be tested:

- Put- vs Call- Volatilities: If it were European options, then the put-call parity should lead to the implied volatilities being equal. However, since those are american options, I would expect them to be close, but not equal.
- Volatilities for different strikes: Due to the shortcomings of black scholes, tail dependence and the like, one would expect the volatilities for different strikes to be different, aka higher for large decreases in the stock price.
- Implied volatilities should be in the range observed in the past. However, since in the volatility notebook I have seen that the past is an imperfect predictor I would be surprised if it were exactly the same. Also it will be interesting to see to what prior window the implied volatilities correspond.


In [21]:
opt_0 = opt[opt["date"] == min(opt["date"])]
opt_1 = opt[opt["date"] == max(opt["date"])]
t_0 = round(min(opt_0["t_n"] * 252))
t_1 = round(min(opt_1["t_n"] * 252))


fig = go.Figure(layout_title="Volatility smile for GOOG",
               layout_xaxis_title="K/S",
               layout_yaxis_title="Implied volatility"
               )
fig.add_trace(go.Scatter(x=opt_0["strike"] / s0, y=opt_0["call_sig"],
                        line={"color": col_0, "dash": "solid"}, name=f"Call-Implied volatility, t:{t_0}d", mode="lines"))
fig.add_trace(go.Scatter(x=opt_0["strike"] / s0, y=opt_0["put_sig"],
                        line={"color": col_0, "dash": "dash"}, name=f"Put-Implied volatility, t:{t_0}d", mode="lines"))
fig.add_trace(go.Scatter(x=opt_1["strike"] / s0, y=opt_1["call_sig"],
                        line={"color": col_1, "dash": "solid"}, name=f"Call-Implied volatility, t:{t_1}d", mode="lines"))
fig.add_trace(go.Scatter(x=opt_1["strike"] / s0, y=opt_1["put_sig"],
                        line={"color": col_1, "dash": "dash"}, name=f"Put-Implied volatility, t:{t_1}d", mode="lines"))
iplot(fig)

There are some remarkable features to be observed here:

**Put- call difference**: Puts have the higher implied volatility (aka are more expensive) than call options (on GOOG). While this does not necessarily follow the theory behind BS, I found plausible explanations explaining this difference by the higher demand for puts ("insurance" against downturns) than for calls. This would mean that the "true" expected volatility lies somewhere in between those two lines. Could an options trader then not make money over the long term by being on the opposide side of those trades? One problem could be that since in crises most asset classes are correlated and losses on puts are unbounded, this could wipe out the trader. Would be interesting to discuss with someone who has knowledge about this. Please contact me if you do and would be willing to discuss!

**Difference between dates**: According to hull (Chapter 20), volatility expectations are mean-reverting which, if the reason for the decrease in implied volatility on the later contract, would mean that the volatility expected between the two expiry dates is lower than the one up to the first expiry date. Without evidence I could also see part of the reason being the end of the year (window dressing?), but without historic data, this is hard to confirm. Unfortunately, I did not find historic option prices freely available online, so these hypotheses cannot really be tested.

**Lack of decreate for calls**: interestingly, the call smile is significantly flatter than the put smile, with the implied volatility even increasing for low strikes. If, as argued for the put-call difference, the implied volatilities reflect supply and demand, then this would mean that the demand for low strikes strongly tails off towards lower and lower strikes.


In [59]:
x = list(range(10, 50))
returns = hist["Close"].pct_change().map(lambda x: np.log(1+x))
y = [returns.iloc[-t:].std() * np.sqrt(252) for t in x]

In [60]:
fig = go.Figure(layout_title="Obseved annualised log volatility in same order of magnitude as implied forward volatility",
               layout_xaxis_title="# Trading days included in vol calculation",
               layout_yaxis_title="Observed volatility"
               )
fig.add_trace(go.Scatter(x=x, y=y,
                        line={"color": col_0, "dash": "solid"}, mode="lines"))

iplot(fig)

I am aware that implied volatilities are forward looking whereas historic changes in stock prices are forward looking, but an order-of-magnitudes comparison still can be comforting. Over "similar" window sizes (aka 28/38 days), the volatilities are very comparable. However, going further back, the volatilities were obviously considerable higher (March 2020 clearly visible). Interesting is the precipitous drop around 40 days. After mid to end October, the price seems to have been rather stable. It appears the markets assume this stability to continue in the near future.